In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2021-03-05 16:23:33,894 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\access.log


In [3]:
%load_ext autoreload 
%autoreload 2
from virasana.integracao.conformidade_alchemy import Conformidade
from virasana.scripts.conformidadeupdate import completa_conformidade, preenche_isocode, preenche_bbox
engine = create_engine(SQL_URI)
# completa_conformidade(db, engine, 200)
# preenche_isocode(db, engine, 200)
# preenche_bbox(db, engine, 20)

# Checagens de conformidade

In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
rs = session.execute('SELECT count(*) FROM dbmercante.ajna_conformidade;')

In [6]:
rs.scalar()

132543

In [7]:
SQL_CONFORMIDADE = '''
SELECT cod_recinto as Recinto, count(*) as "Qtde de imagens",
 CAST(avg(height) AS INT) as "Linhas (média)",
 CAST(avg(width) AS INT) as "Colunas (média)",
 avg(ratio) as "Relação largura/altura (média)",
 SUM(height < 700) / count(*) * 100 as "% Tamanho pequeno",
 SUM(ratio < 1.5) / count(ratio) * 100 as "% relação abaixo 1.5",
 avg(laplacian) as "Índice de nitidez ou ruído",
 avg(bbox_score) *100 as "% Confiança da VC",
 (sum(bbox_classe>=3) / count(*)) * 100 as "% VC não encontra CC"
 from ajna_conformidade 
 group by cod_recinto
 '''

In [74]:
df = pd.read_sql(SQL_CONFORMIDADE, engine)

In [76]:
pd.options.display.float_format = '{:,.1f}'.format
df

,Recinto,Qtde de imagens,Linhas (média),Colunas (média),Relação largura/altura (média),% Tamanho pequeno,% relação abaixo 1.5,Índice de nitidez ou ruído,% Confiança da VC,% VC não encontra CC
0,BANDEIRANTES,1780,799,1920,2.4,0.0,4.0,78.9,95.3,2.3
1,BTP1,28219,799,2126,2.7,0.0,0.4,69.4,99.3,0.2
2,BTP2,13400,799,1861,2.3,0.0,1.2,88.8,97.9,0.8
3,CRAGEA,720,880,1921,2.2,0.0,7.8,171.2,98.4,0.1
4,DEICMAR,4238,799,1344,1.7,0.0,32.0,144.2,67.8,11.8
5,ECOPORTO,5238,799,2134,2.7,0.1,1.1,76.9,99.0,0.4
6,EMBRAPORT,17206,799,3687,4.6,0.0,0.0,396.3,97.9,1.2
7,EUDMARCO,2234,234,883,3.9,92.8,0.0,885.6,99.9,0.0
8,LOCALFRIO,5554,799,1761,2.2,0.0,1.5,86.7,97.8,0.8
9,MARIMEX,5836,799,2038,2.6,0.0,1.0,79.1,99.3,0.3


In [77]:
df.values

array([['BANDEIRANTES', 1780, 799, 1920, 2.401938, 0.0, 3.9888, 78.87,
        95.269058, 2.3034],
       ['BTP1', 28219, 799, 2126, 2.66036, 0.0, 0.3756, 69.3545,
        99.279239, 0.2091],
       ['BTP2', 13400, 799, 1861, 2.32879, 0.0, 1.1791, 88.8297,
        97.880769, 0.7985],
       ['CRAGEA', 720, 880, 1921, 2.183694, 0.0, 7.7778, 171.1905,
        98.412698, 0.1389],
       ['DEICMAR', 4238, 799, 1344, 1.681654, 0.0, 31.9962, 144.2227,
        67.774131, 11.8216],
       ['ECOPORTO', 5238, 799, 2134, 2.672073, 0.0573, 1.05, 76.8807,
        99.00232, 0.3627],
       ['EMBRAPORT', 17206, 799, 3687, 4.613713, 0.0, 0.0058, 396.251,
        97.920768, 1.2031],
       ['EUDMARCO', 2234, 234, 883, 3.929624, 92.838, 0.0, 885.5676,
        99.936224, 0.0],
       ['LOCALFRIO', 5554, 799, 1761, 2.204316, 0.0, 1.4584, 86.6697,
        97.846778, 0.8462],
       ['MARIMEX', 5836, 799, 2038, 2.55019, 0.0, 0.9596, 79.1146,
        99.316541, 0.2742],
       ['NOVALOG', 3429, 799, 1769, 2.

In [78]:
df.keys()

Index(['Recinto', 'Qtde de imagens', 'Linhas (média)', 'Colunas (média)',
       'Relação largura/altura (média)', '% Tamanho pequeno',
       '% relação abaixo 1.5', 'Índice de nitidez ou ruído',
       '% Confiança da VC', '% VC não encontra CC'],
      dtype='object')

In [15]:
rs = session.execute(SQL_CONFORMIDADE)

In [16]:
rs.keys()

['Recinto',
 'Qtde de imagens',
 'Linhas (média)',
 'Colunas (média)',
 'Relação largura/altura (média)',
 '% Tamanho pequeno',
 '% relação abaixo 1.5',
 'Índice de nitidez ou ruído',
 '% Confiança da VC',
 '% VC não encontra CC']

In [17]:
for row in rs:
    print([str(col) for col in row])


['BANDEIRANTES', '1780', '799', '1920', '2.401938', '0.0000', '3.9888', '78.8700', '95.269058', '2.3034']
['BTP1', '28219', '799', '2126', '2.660360', '0.0000', '0.3756', '69.3545', '99.279239', '0.2091']
['BTP2', '13400', '799', '1861', '2.328790', '0.0000', '1.1791', '88.8297', '97.880769', '0.7985']
['CRAGEA', '720', '880', '1921', '2.183694', '0.0000', '7.7778', '171.1905', '98.412698', '0.1389']
['DEICMAR', '4238', '799', '1344', '1.681654', '0.0000', '31.9962', '144.2227', '67.774131', '11.8216']
['ECOPORTO', '5238', '799', '2134', '2.672073', '0.0573', '1.0500', '76.8807', '99.002320', '0.3627']
['EMBRAPORT', '17206', '799', '3687', '4.613713', '0.0000', '0.0058', '396.2510', '97.920768', '1.2031']
['EUDMARCO', '2234', '234', '883', '3.929624', '92.8380', '0.0000', '885.5676', '99.936224', '0.0000']
['LOCALFRIO', '5554', '799', '1761', '2.204316', '0.0000', '1.4584', '86.6697', '97.846778', '0.8462']
['MARIMEX', '5836', '799', '2038', '2.550190', '0.0000', '0.9596', '79.1146', '